### Imports

In [ ]:
%load_ext autoreload
%autoreload 2

from astropy.constants import si as constants
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm 
from survey_tools import sky
from survey_tools.utility import plot

### Parameters

In [ ]:
instrument        = 'ERIS'
location          = 'Paranal' # MaunaKea, Paranal
resolving_power   = 5000

target_redshift   = 0.8732
target_radius     = 1.0    # arcsec
target_flux_Ha    = 50e-17 # erg/s/cm^2
target_velocity_dispersion = 200 # km/s

airmass           = 2.0
min_photon_rate   = 10   # ph/s/arcsec^2/nm/m^2
trans_min         = 0.50 # percent
trans_multiple    = 0.5  # FWHM of Ha
avoid_multiple    = 0.5  # FWHM of Ha
wavelength_range  = np.array([[10900, 14200], [14500, 18700], [19300,24800]]) # angstrom
flux_NII_Ha_ratio = 0.10

### Load Sky Transmission

In [ ]:
sky_transmission_data = sky.load_transmission_data(location, airmass)
print(f"Number of Sky Transmission points: {len(sky_transmission_data)}, dLambda = {sky_transmission_data['wavelength'][1]-sky_transmission_data['wavelength'][0]:.3f} A")

### Load Sky Background

In [ ]:
sky_background_data = sky.load_background_data(location, airmass)
print(f"Number of Sky Background points: {len(sky_background_data)}, dLambda = {sky_background_data['wavelength'][1]-sky_background_data['wavelength'][0]:.3f} A")

### Export Sky Lines

In [9]:
for i in range(len(wavelength_range)):
    sky_background_data_low_res = sky.get_low_res_background(sky_background_data, np.array(wavelength_range[i]), resolving_power)
    sky_lines = sky.find_sky_lines(sky_background_data_low_res, 10)
    sky_lines.write(f"../output/skylines-{location}-{instrument}-R{resolving_power}-{wavelength_range[i][0]}-{wavelength_range[i][1]}.txt", format='ascii', overwrite=True)

### Compute Target Quantities

In [ ]:
wavelength_Ha = 6564.610 * (1 + target_redshift)
wavelength_Ha_atm = sky.get_vacuum_to_air_wavelength(wavelength_Ha)
wavelength_NIIa = 6549.89  * (1 + target_redshift)
wavelength_NIIa_atm = sky.get_vacuum_to_air_wavelength(wavelength_NIIa)
wavelength_NIIb = 6585.27  * (1 + target_redshift)
wavelength_NIIb_atm = sky.get_vacuum_to_air_wavelength(wavelength_NIIb)

SB_Ha = target_flux_Ha/(np.pi*target_radius**2) # erg/s/cm^2/arcsec^2
FWHM_Ha = wavelength_Ha_atm * target_velocity_dispersion / constants.c.to('km/s').value

dwavelength    = wavelength_Ha_atm / resolving_power
dwavelength_Ha = np.sqrt(dwavelength**2 + FWHM_Ha**2)
sigma_Ha   = dwavelength_Ha / 2.35482 # FWHM -> sigma

ph_energy  = 6.626e-27 * 2.998e10 / (wavelength_Ha_atm/1e8) # erg
ph_rate_Ha = SB_Ha / ph_energy / (FWHM_Ha/10) * 100**2  # ph/s/arcsec^2/nm/m^2

print(f"    Redshift: {target_redshift:.4f}")
print(f"Ha in Vacuum: {wavelength_Ha/10:.2f} nm")
print(f"         Atm: {wavelength_Ha_atm/10:.2f} nm")
print(f"        FWHM: {FWHM_Ha/10:.2f} nm [{target_velocity_dispersion:.0f} km/s]")
print(f"           R: {resolving_power:.0f} [dLambda = {dwavelength/10:.1f} nm]")
print(f"   Line Flux: {target_flux_Ha:.1e} erg/s/cm^2")
print(f"          SB: {SB_Ha:.1e} erg/s/cm^2/arcsec^2  [{SB_Ha/1000:.1e} W/m^2/arcsec^2    ]")
print(f"     Ph Rate: {ph_rate_Ha:.1f}     ph/s/arcsec^2/nm/m^2 [{ph_rate_Ha*ph_energy*1e-7*1e9:.1e} J/s/m^2/m/arcsec^2]")

sky_rate = sky.get_background(sky_background_data, wavelength_Ha_atm, [wavelength_Ha_atm - FWHM_Ha*10, wavelength_Ha_atm + FWHM_Ha*10], resolving_power)
print(f" Sky Ph Rate: {sky_rate:.1f}     ph/s/arcsec^2/nm/m^2 [{sky_rate*ph_energy*1e-7*1e9:.1e} J/s/m^2/m/arcsec^2]")

trans_Ha = sky.get_mean_transmission(sky_transmission_data, wavelength_Ha_atm, target_velocity_dispersion, resolving_power, trans_multiple)
print(f"  Mean Trans: {trans_Ha*100:.1f}%")

del dwavelength, SB_Ha, FWHM_Ha, ph_energy, sky_rate, trans_Ha

### Compare to ERIS ETC

Source:
  Flux     = 10e-17 erg/s/cm^2
  Diameter = 1 arcsec
  SB       = 6.4e-17 erg/s/cm^2/arcsec^2 = 6.4e-20 erg/s/cm^2/arcsec^2

Configure ETC as follows:
* Extended Source Infinite, Emission Line at Sky Line Wavelength, FWHM=0.82nm (200km/s), Flux = 6.4e-20 W/m2/arcsec2
* Airmass=1.9, FLI=0.5, pwv=30.00 (worst case permitted for LGS-AO)
* LGS AO, 50% seeing, NGS 12 0" (default), Extract 3px
* IFS, J_low, 100mas
* DIT = 300s x NDIT = 36 = 3h exposure

Use sky background for airmass=2

Test cases (Target = ETC 7.3e-11 J/s/m^2/m/arcsec^2):
1. No Skyline at 1253.76nm [MK:  0.8 ph/s/arcsec^2/nm/m^2, ETC: 7.2e-11 J/s/m^2/m/arcsec^2, 95% atm trans]: S/N=3.79
2.    Skyline at 1221.12nm [MK:  5.8 ph/s/arcsec^2/nm/m^2, ETC: 8.3e-10 J/s/m^2/m/arcsec^2, 96% atm trans]: S/N=3.33 -12%
3.    Skyline at 1250.96nm [MK: 12.5 ph/s/arcsec^2/nm/m^2, ETC: 2.7e-09 J/s/m^2/m/arcsec^2, 93% atm trans]: S/N=3.03 -20%
4.    Skyline at 1248.24nm [MK: 27.0 ph/s/arcsec^2/nm/m^2, ETC: 4.1e-09 J/s/m^2/m/arcsec^2, 96% atm trans]: S/N=2.67 -28%

To limit drop to <20%, place threshold between #2 and #3 so 10 ph/s/arcsec^2/nm/m^2

In [ ]:
compare_wavelength  = 1253.76 # nm
compare_dwavelength = 20.0    # nm
sky_background_data_low_res = sky.get_low_res_background(sky_background_data, np.array([compare_wavelength-compare_dwavelength/2, compare_wavelength+compare_dwavelength/2])*10, resolving_power)
mid_idx = len(sky_background_data_low_res) // 2

_, ax = plot.create_plot(title=f"{location} IR Sky Background")
ax.plot(sky_background_data_low_res['wavelength']/10, sky_background_data_low_res['emission'], linestyle='-', color='b', linewidth=1)
ax.set_xlabel('Wavelength [$nm$]')
ax.set_ylabel('Emission [$ph/s/arcsec^2/nm/m^2$]')
ax.axvline(sky_background_data_low_res['wavelength'][mid_idx]/10, linestyle=':', linewidth=1, color='k')

print(f"Sky Background at {sky_background_data_low_res['wavelength'][mid_idx]/10:.2f} nm = {sky_background_data_low_res['emission'][mid_idx]:.1f} ph/s/arcsec^2/nm/m^2")

del compare_wavelength, compare_dwavelength, sky_background_data_low_res, mid_idx, ax

### Plot Sky Tranmission

In [ ]:
if np.ndim(wavelength_range) == 1:
    min_wavelength = wavelength_range[0]
    max_wavelength = wavelength_range[1]
else:
    min_wavelength = wavelength_range[0,0]
    max_wavelength = wavelength_range[-1,1]

plot_xrange_start = [min_wavelength, wavelength_NIIa_atm-500, wavelength_NIIa_atm-25]
plot_xrange_end   = [max_wavelength, wavelength_NIIb_atm+500, wavelength_NIIb_atm+25]

plot_yrange_start = [0.0, 0.5, 0.7]
plot_yrange_end   = [1.0, 1.0, 1.0]

lgray = (0.5,0.5,0.5)
alpha = 0.3

for i in np.arange(len(plot_xrange_start)):
    wavelength_filter = (sky_transmission_data['wavelength'] >= plot_xrange_start[i]) & (sky_transmission_data['wavelength'] <= plot_xrange_end[i])

    wavelength = sky_transmission_data['wavelength'][wavelength_filter]
    dwavelength = wavelength / resolving_power

    _, ax = plot.create_plot(title=f"{location} IR Sky Transmission")
    ax.plot(wavelength, sky_transmission_data['transmission'][wavelength_filter], linestyle='-', color='b', linewidth=1)
    ax.set_xlabel('Wavelength [Angstrom]')
    ax.set_ylabel('Transmission [%]')
    ax.set_xlim([plot_xrange_start[i], plot_xrange_end[i]])
    ax.set_ylim([plot_yrange_start[i], plot_yrange_end[i]])
    if wavelength_Ha_atm > plot_xrange_start[i] and wavelength_Ha_atm < plot_xrange_end[i]:
        trans_NIIa = sky.get_mean_transmission(sky_transmission_data, wavelength_NIIa_atm, target_velocity_dispersion, resolving_power, trans_multiple)
        colour = 'r' if trans_NIIa < trans_min else 'g'
        ax.fill_between(wavelength, plot_yrange_start[i], plot_yrange_end[i], where=(abs(wavelength - wavelength_NIIa_atm) < dwavelength_Ha*trans_multiple), facecolor=lgray, alpha=alpha)
        plt.hlines(y=trans_NIIa, xmin=(wavelength_NIIa_atm - dwavelength_Ha*trans_multiple), xmax=(wavelength_NIIa_atm + dwavelength_Ha*trans_multiple), linestyle='-', linewidth=2, color=colour)

        trans_Ha = sky.get_mean_transmission(sky_transmission_data, wavelength_Ha_atm, target_velocity_dispersion, resolving_power, trans_multiple)
        colour = 'r' if trans_Ha < trans_min else 'g'
        ax.fill_between(wavelength, plot_yrange_start[i], plot_yrange_end[i], where=(abs(wavelength - wavelength_Ha_atm) < dwavelength_Ha*trans_multiple), facecolor=lgray, alpha=alpha)
        plt.hlines(y=trans_Ha, xmin=(wavelength_Ha_atm - dwavelength_Ha*trans_multiple), xmax=(wavelength_Ha_atm + dwavelength_Ha*trans_multiple), linestyle='-', linewidth=2, color=colour)

        trans_NIIb = sky.get_mean_transmission(sky_transmission_data, wavelength_NIIb_atm, target_velocity_dispersion, resolving_power, trans_multiple)
        colour = 'r' if trans_NIIb < trans_min else 'g'
        ax.fill_between(wavelength, plot_yrange_start[i], plot_yrange_end[i], where=(abs(wavelength - wavelength_NIIb_atm) < dwavelength_Ha*trans_multiple), facecolor=lgray, alpha=alpha)
        plt.hlines(y=trans_NIIb, xmin=(wavelength_NIIb_atm - dwavelength_Ha*trans_multiple), xmax=(wavelength_NIIb_atm + dwavelength_Ha*trans_multiple), linestyle='-', linewidth=2, color=colour)

del plot_xrange_start, plot_xrange_end, plot_yrange_start, plot_yrange_end, lgray, alpha
del trans_Ha, trans_NIIa, trans_NIIb
del wavelength_filter, wavelength, dwavelength, colour, i, ax

### Plot Sky Background

In [ ]:
plot_log = True
plot_xrange_start = [min_wavelength, wavelength_NIIa_atm-500, wavelength_NIIa_atm-25]
plot_xrange_end   = [max_wavelength, wavelength_NIIb_atm+500, wavelength_NIIb_atm+25]
ylim = [1e-1, 1e4]

lgray = (0.5,0.5,0.5)
alpha = 0.3

for i in np.arange(len(plot_xrange_start)):
    plot_wavelength_range = [plot_xrange_start[i], plot_xrange_end[i]]
    wavelengths = np.linspace(plot_wavelength_range[0], plot_wavelength_range[1], 1000)
    sky_background_data_low_res = sky.get_low_res_background(sky_background_data, plot_wavelength_range, resolving_power)
    sky_lines = sky.find_sky_lines(sky_background_data_low_res, min_photon_rate)

    wavelength = sky_background_data_low_res['wavelength']
    _, ax = plot.create_plot(title=f"{location} IR Sky Background")
    ax.plot(wavelength, sky_background_data_low_res['emission'], linestyle='-', color='b', linewidth=1)
    ax.scatter(sky_lines['wavelength'], sky_lines['emission'], marker='x', color='b')
    if len(sky_lines) <= 5:
        plt.hlines(y=sky_lines["width_height"], xmin=sky_lines["wavelength_low"], xmax=sky_lines["wavelength_high"], linestyle='-', color='b')        
    ax.axhline(min_photon_rate, linestyle=':', linewidth=1, color='k')
    ax.set_xlabel('Wavelength [Angstrom]')
    ax.set_ylabel('Emission [$ph/s/arcsec^2/nm/m^2$]')
    ax.set_xlim(plot_wavelength_range)
    if plot_log:
        ax.set_yscale('log')
        ax.set_ylim(ylim)

    if wavelength_NIIa_atm > plot_xrange_start[i] and wavelength_NIIa_atm < plot_xrange_end[i]:
        colour = 'r' if sky.reject_emission_line(sky_background_data, sky_transmission_data,wavelength_NIIa_atm, target_velocity_dispersion,  resolving_power, wavelength_range, trans_min, trans_multiple, avoid_multiple, min_photon_rate) else 'g'
        ax.fill_between(wavelength, ylim[0], ylim[1], where=(abs(wavelength - wavelength_NIIa_atm) < dwavelength_Ha*avoid_multiple), facecolor=lgray, alpha=alpha)
        ax.plot(wavelengths, ph_rate_Ha * flux_NII_Ha_ratio * np.sqrt(2*np.pi) * sigma_Ha * norm.pdf(wavelengths, wavelength_NIIa_atm, sigma_Ha), linestyle='-', linewidth=2, color=colour)
        if avoid_multiple != 0.5:
            ax.fill_between(wavelength, ylim[0], ylim[1], where=(abs(wavelength - wavelength_NIIa_atm) < dwavelength_Ha/2), facecolor=lgray, alpha=alpha)
        #plt.hlines(y=(PF_Ha * flux_NII_Ha_ratio)/2, xmin=(lambda_NIIa_atm-dLambda_Ha/2), xmax=(lambda_NIIa_atm+dLambda_Ha/2), linestyle='-', linewidth=2, color=colour)        

    if wavelength_Ha_atm > plot_xrange_start[i] and wavelength_Ha_atm < plot_xrange_end[i]:
        colour = 'r' if sky.reject_emission_line(sky_background_data, sky_transmission_data, wavelength_Ha_atm, target_velocity_dispersion, resolving_power, wavelength_range, trans_min, trans_multiple, avoid_multiple, min_photon_rate) else 'g'
        ax.fill_between(wavelength, ylim[0], ylim[1], where=(abs(wavelength - wavelength_Ha_atm) < dwavelength_Ha*avoid_multiple), facecolor=lgray, alpha=alpha)
        ax.plot(wavelengths, ph_rate_Ha * np.sqrt(2*np.pi) * sigma_Ha * norm.pdf(wavelengths, wavelength_Ha_atm, sigma_Ha), linestyle='-', linewidth=2, color=colour)
        if avoid_multiple != 0.5:
            ax.fill_between(wavelength, ylim[0], ylim[1], where=(abs(wavelength - wavelength_Ha_atm) < dwavelength_Ha/2), facecolor=lgray, alpha=alpha)
        #plt.hlines(y=PF_Ha/2, xmin=(lambda_Ha_atm-dLambda_Ha/2), xmax=(lambda_Ha_atm+dLambda_Ha/2), linestyle='-', linewidth=2, color=colour)        

    if wavelength_NIIb_atm > plot_xrange_start[i] and wavelength_NIIb_atm < plot_xrange_end[i]:
        colour = 'r' if sky.reject_emission_line(sky_background_data, sky_transmission_data, wavelength_NIIb_atm, target_velocity_dispersion, resolving_power, wavelength_range, trans_min, trans_multiple, avoid_multiple, min_photon_rate) else 'g'
        ax.fill_between(wavelength, ylim[0], ylim[1], where=(abs(wavelength - wavelength_NIIb_atm) < dwavelength_Ha*avoid_multiple), facecolor=lgray, alpha=alpha)
        ax.plot(wavelengths, ph_rate_Ha * flux_NII_Ha_ratio * np.sqrt(2*np.pi) * sigma_Ha * norm.pdf(wavelengths, wavelength_NIIb_atm, sigma_Ha), linestyle='-', linewidth=2, color=colour)
        if avoid_multiple != 0.5:
            ax.fill_between(wavelength, ylim[0], ylim[1], where=(abs(wavelength - wavelength_NIIb_atm) < dwavelength_Ha/2), facecolor=lgray, alpha=alpha)
        #plt.hlines(y=(PF_Ha * flux_NII_Ha_ratio)/2, xmin=(lambda_NIIb_atm-dLambda_Ha/2), xmax=(lambda_NIIb_atm+dLambda_Ha/2), linestyle='-', linewidth=2, color=colour)        

    if i == 0:
        emission_no_peaks = sky_background_data_low_res['emission'].copy()
        for j in np.arange(len(sky_lines)):
            emission_no_peaks[(sky_background_data_low_res['wavelength'] >= (sky_lines['wavelength_low'][j] - dwavelength_Ha * avoid_multiple)) \
                            & (sky_background_data_low_res['wavelength'] <= (sky_lines['wavelength_high'][j] + dwavelength_Ha * avoid_multiple))] = None

        _, ax = plot.create_plot(title=f"{location} IR Sky Background")
        ax.plot(sky_background_data_low_res['wavelength'], emission_no_peaks, linestyle='-', color='b', linewidth=1)
        ax.axhline(min_photon_rate, linestyle='--', linewidth=1, color='k')
        ax.set_xlabel('Wavelength [Angstrom]')
        ax.set_ylabel('Emission [$ph/s/arcsec^2/nm/m^2$]')
        ax.set_xlim(plot_wavelength_range)
        if plot_log:
            ax.set_yscale('log')
            ax.set_ylim([1e-1, min_photon_rate+10])
        else:
            ax.set_ylim([0, min_photon_rate+0.25])

del plot_log, plot_xrange_start, plot_xrange_end, ylim, lgray, alpha, plot_wavelength_range
del sky_background_data_low_res, sky_lines, emission_no_peaks
del wavelengths, wavelength, i, j, colour, ax